In [1]:
import sys
import os
import pandas as pd
import numpy as np
import json
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Edit this: relative path to directory containing exported data
path = 'export/Default Term/POC/'

In [3]:
# Get the list of all json and attachment files in directory tree
listOfJSON = list()
listOfAttachments = list()
for (dirpath, dirnames, filenames) in os.walk(os.getcwd() + '/' + path):
    relpath = dirpath.replace(os.getcwd() + '/','')
    listOfJSON += [[relpath, file] for file in filenames if file.endswith('.json')]
    listOfAttachments += [[relpath, file] for file in filenames if not file.endswith('.json')]

# Get the JSON data
jsons = []
attachments, comments, comment_attachments, rubric_assessment = [], [], [], []
for (dirpath, jsonfile) in listOfJSON:
    if jsonfile.endswith('.json'):
        with open(os.path.join(dirpath, jsonfile)) as f:
            js = json.load(f)
            jsons.append(js)
            #Get submission attachments
            attachment_list = ''
            if 'attachments' in js:
                attachment_list = [att['filename'] for att in js['attachments']]
            attachments.append(attachment_list)
            #Get submission comments and any attachment
            comms = ''
            comm_attachments = ''
            if 'submission_comments' in js:
                comms = [comm for comm in js['submission_comments']]
                for comm_attach in js['submission_comments']:
                    if 'attachments' in comm_attach:
                        comm_attachments = [comm['filename'] for comm in comm_attach['attachments'] ]
            comments.append(comms or '') #or: do not store []
            comment_attachments.append(comm_attachments or '')
            #Get full_rubric_assessments
            r_assess = ''
            if 'full_rubric_assessment' in js:
                r_assess = js['full_rubric_assessment']
            rubric_assessment.append(r_assess)
                            

(dirpaths, jsonfiles) = zip(*listOfJSON)

# Store all data in a DataFrame. Infer more metadata
df = pd.DataFrame({'path':dirpaths, 'file':jsonfiles, 'json': jsons, 
                   'attachments': attachments, 'comments': comments, 
                   'comment_attachments': comment_attachments, 'rubric_assessment': rubric_assessment})
df['id'] = df.json.apply(lambda json: json['id'])
df['type'] = df.file.apply(lambda json: json.split('_')[0])
hierarchy = ['account','course','assignment','submission']
df['parent'] = df.apply(lambda x: x.json[hierarchy[hierarchy.index(x['type'])-1] + '_id'], axis=1)


df #.loc[6,'attachments']

,path,file,json,attachments,comments,comment_attachments,rubric_assessment,id,type,parent
0,export/Default Term/POC/,course_1788710.json,"{'account_id': 81259, 'calendar': {'ics': 'htt...",,,,,1788710,course,81259
1,export/Default Term/POC/assignment_13280399,assignment_13280399.json,"{'has_overrides': False, 'max_name_length': 25...",,,,,13280399,assignment,1788710
2,export/Default Term/POC/assignment_13280399/su...,submission_207012045.json,"{'grade': None, 'grading_period_id': None, 'co...",,,,,207012045,submission,13280399
3,export/Default Term/POC/assignment_13280399/su...,submission_207012273.json,"{'grade': None, 'grading_period_id': None, 'co...",,,,,207012273,submission,13280399
4,export/Default Term/POC/assignment_13280399/su...,submission_207010516.json,"{'grade': None, 'grading_period_id': None, 'co...",,,,,207010516,submission,13280399
5,export/Default Term/POC/assignment_13280576,assignment_13280576.json,"{'use_rubric_for_grading': False, 'has_overrid...",,,,,13280576,assignment,1788710
6,export/Default Term/POC/assignment_13280576/su...,submission_207012274.json,"{'attachments': [{'hidden_for_user': False, 'u...","[oblig3.py, oblig3-4.pdf]","[{'comment': 'There u go :)', 'edited_at': Non...",[lunaMoth.jpg],"{'artifact_id': 207012274, 'artifact_attempt':...",207012274,submission,13280576
7,export/Default Term/POC/assignment_13280576/su...,submission_207010999.json,"{'attachments': [{'hidden_for_user': False, 'u...","[test.py, test.pdf]","[{'comment': 'This is my submission', 'edited_...",,"{'artifact_id': 207010999, 'artifact_attempt':...",207010999,submission,13280576
8,export/Default Term/POC/assignment_13280576/su...,submission_207012046.json,"{'grade': None, 'grading_period_id': None, 'co...",,,,,207012046,submission,13280576
9,export/Default Term/POC/assignment_13280375,assignment_13280375.json,"{'use_rubric_for_grading': True, 'has_override...",,,,,13280375,assignment,1788710


In [4]:
# Print IDs of all courses, assignments, submission, comments
print('Courses:')
df.loc[df.type == 'course'].id
print('Assignments:')
df.loc[df.type == 'assignment'].id
print('Submissions:')
df.loc[df.type == 'submission'].id
print('Comments:')
df.loc[df.type == 'comment'].id

Courses:


0    1788710
Name: id, dtype: int64

Assignments:


1    13280399
5    13280576
9    13280375
Name: id, dtype: int64

Submissions:


2     207012045
3     207012273
4     207010516
6     207012274
7     207010999
8     207012046
10    207012272
11    207012044
12    207010515
Name: id, dtype: int64

Comments:


Series([], Name: id, dtype: int64)

In [5]:
# Print all rubrics
def print_rubric(df, assignment_id):
    rubric = df.loc[(df.id == assignment_id)].json.iloc[0]['rubric']
    print('\033[1mRubric for assignment %s\033[0m' % assignment_id)
    template = "| {0:6} | {1:32} | {2:8} | {3:40} |"
    print(template.format('points', 'description', 'id', 'long_description'))
    
    for r_i, r_item in enumerate(rubric):
        for r_rating in r_item['ratings']:
            #r_rating
            #print('%s - %s - %s - %s' % (r_rating['points'], r_rating['description'], r_rating['id'], r_rating['long_description']))
            print(template.format(r_rating['points'], r_rating['description'], r_rating['id'], r_rating['long_description']))
            #rubric[0]['ratings']
    print('\n'.rjust(len(template.format(0,0,0,0)),'-'))


for asgn in df[df.type == 'assignment'].json:
    if 'rubric' in asgn:
        #print(asgn['id'], '\n', asgn['rubric'])
        print_rubric(df, asgn['id'])

Rubric for assignment 13280576
| points | description                      | id       | long_description                         |
|    5.0 | Great syntax                     | blank    |                                          |
|    0.0 | Bad syntax                       | blank_2  |                                          |
|   50.0 | Force vector is correct          | _9641    |                                          |
|    0.0 | Force vector is bad              | _7790    |                                          |
--------------------------------------------------------------------------------------------------

Rubric for assignment 13280375
| points | description                      | id       | long_description                         |
|   20.0 | Full Marks                       | blank    |                                          |
|    0.0 | No Marks                         | blank_2  |                                          |
|   25.0 | Full Marks                 

In [47]:
# Print submission (attachments, comments, full rubric assessments)
for i,row in df.loc[df.rubric_assessment != ''].iterrows():
    ra = row['rubric_assessment']
    print("\033[1mAssignment %d, student %d\033[0m" % (ra['rubric_association']['association_id'], row.json['user_id']))
    
    # Attachments
    print('Attachments: %s' % ', '.join(row.attachments))
    
    # Comments
    print('Comments:',len(row.comments))
    template = "  | {0:30} | {1:10} | {2:20} |"
    print(template.format("comment", "author_id", "attachments"))
    for comment in row.comments:
        print(template.format(comment['comment'], str(comment['author_id']), ', '.join(row.loc['comment_attachments'])))
    print(' ',''.rjust(70,'-'))
    
    # Full rubric assessment
    print("Full rubric assessment:")
    template = "  | {0:6} | {1:32} | {2:50} |"
    print(template.format("Points", "Description", "Comment"))
    for data in ra['data']:        
        for rec in [(data['points'], data['description'], data['comments'])]: 
            print(template.format(*rec))
    print(' ','\n'.rjust(98,'-'))

Assignment 13280576, student 25104805
Attachments: oblig3.py, oblig3-4.pdf
Comments: 2
  | comment                        | author_id  | attachments          |
  | There u go :)                  | 25104805   | lunaMoth.jpg         |
  | There you are                  | 23434396   | lunaMoth.jpg         |
  ----------------------------------------------------------------------
Full rubric assessment:
  | Points | Description                      | Comment                                            |
  |    0.0 | No Marks                         | Too bad!                                           |
  |   50.0 | Full Marks                       | Nice                                               |
  -------------------------------------------------------------------------------------------------

Assignment 13280576, student 23605017
Attachments: test.py, test.pdf
Comments: 1
  | comment                        | author_id  | attachments          |
  | This is my submission          | 23